In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
llm = ChatOpenAI()

## Create Tools

In [18]:
from langchain_core.tools import InjectedToolArg
from typing import Type, Annotated

In [19]:
@tool
def get_conversion_factor(base_currency:str, target_currency:str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency 
    and the target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/184598f9344b3cc756393583/pair/{base_currency}/{target_currency}"
    
    response = requests.get(url)
    return response.json()

In [20]:
get_conversion_factor.invoke({"base_currency":"USD", "target_currency":"INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745712001,
 'time_last_update_utc': 'Sun, 27 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745798401,
 'time_next_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4301}

In [ ]:

@tool
def convert(base_currency:int, conversion_rate:Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate, this function converts the target currency value from given 
    base currency value

    """
    return base_currency * conversion_rate

In [23]:
convert.invoke({"base_currency": 10,  "conversion_rate": 85.1})

851.0

### Tool Binding

In [24]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [39]:
messages = [HumanMessage('What is conversion factor between USD and INR? and based on that can you convert 10 USD to INR?')]
messages

[HumanMessage(content='What is conversion factor between USD and INR? and based on that can you convert 10 USD to INR?', additional_kwargs={}, response_metadata={})]

In [40]:
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jaKtmik9GWO6nQgTRKhNejBK', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_d90R0WAHCN5usXKYWpsdJEs6', 'function': {'arguments': '{"base_currency": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 125, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fa69723c-702b-4cf0-be61-54ac8fbd0154-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_jaKtmik9GWO6nQgTRKhNejBK', '

In [41]:
messages.append(result)

In [42]:
result.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_jaKtmik9GWO6nQgTRKhNejBK',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 10},
  'id': 'call_d90R0WAHCN5usXKYWpsdJEs6',
  'type': 'tool_call'}]

In [43]:
import json

for tool_call in result.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [44]:
messages

[HumanMessage(content='What is conversion factor between USD and INR? and based on that can you convert 10 USD to INR?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jaKtmik9GWO6nQgTRKhNejBK', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_d90R0WAHCN5usXKYWpsdJEs6', 'function': {'arguments': '{"base_currency": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 125, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fa69723c-702b-4cf0-b

In [45]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 85.4301. \n\nBased on that, 10 USD is equivalent to 854.30 INR.'